# Práctica 1 - CART Splitting
El objetivo de esta práctica consiste en implemetar el algoritmo de CART Splitting, el cuál se basa en iterar por las variables del conjunto de entrada de forma que se minimice la siguiente expresión:

![Figura 1. Expresión a minimizar](./img/expresion.png)

## Algoritmo
El algoritmo para minimizar esta suma es el siguiente:
1. Elegimos un valor de X1 que no sea ni el primero ni el último. Ese valor determinará las regiones (s).
2. Aplicamos la suma con ese valor de X1.
3. Repetimos para todos los valores de X1.
4. El valor que nos dé la suma mínima será el que elegiremos.

## Código
Primero, cargamos el dataframe que vamos a usar para crear el árbol

In [6]:
library(tidyverse)
df <- read.csv("./datasets/datos.csv")
creditcard_2023 <- read.csv("./datasets/creditcard_2023.csv")
creditcard_test <- read.csv("./datasets/creditcard_test.csv")
creditcard_test <- creditcard_test[, -1]
cd_size <- dim(creditcard_2023)[1] / 4
creditcard_2023 <- creditcard_2023[, -1]
creditcard_2023
df

Posteriormente, creamos la función que implementa el algoritmo en cuestión.

In [3]:
cart_splitting <- function(data, class_variable,
                           verbose = FALSE) {
  data <- data.frame(lapply(data, sort))
  data_size <- dim(data)[1]
  min_global_sum <- Inf
  varnames <- names(data)[names(data) != class_variable]
  best_var <- NA
  best_s <- Inf
  for (var in varnames) {
    min_sum <- Inf
    for (i in 2:(data_size - 1)) {
      aux_s <- data[[var]][i]
      y_vector_r1 <- data[[class_variable]][data[[var]] < aux_s]
      y_vector_r2 <- data[[class_variable]][data[[var]] >= aux_s]
      aux_sum <- sum((y_vector_r1 - mean(y_vector_r1))^2)
      + sum((y_vector_r2 - mean(y_vector_r2))^2)

      if (aux_sum <= min_sum) {
        min_sum <- aux_sum
        s <- aux_s
      }
      if (verbose) {
        print(sprintf("Variable: %f; Valor: %f; Suma mínima: %f", aux_s,
                      data[[class_variable]][data[[var]] == aux_s],
                      aux_sum))
      }
    }
    if (min_sum < min_global_sum) {
      min_global_sum <- min_sum
      best_s <- s
      best_var <- var
    }
    if (min_global_sum == 0) {
      break
    }
  }
  r1 <- data[data[[best_var]] < best_s, ]
  r2 <- data[data[[best_var]] >= best_s, ]
  return(list(R1 = r1, R2 = r2, value = best_s,
              best_var = best_var, min_sum = min_global_sum))
}

Una vez encontrado el valor que garantiza la suma mínima, obtenemos de la lista resultado las regiones, la suma mínima y el valor de *s*.

In [4]:
split_df <- cart_splitting(df, "y")
split_creditcard_2023 <- cart_splitting(creditcard_2023, "Class")
split_df
split_creditcard_2023

,x1,x2,y
,<dbl>,<dbl>,<int>
1,1.728571,0.4766834,0
2,2.771245,1.1697614,0
3,2.999209,1.7847839,0
4,3.678320,2.2090142,0
5,3.961043,2.6199503,0
,x1,x2,y
,<dbl>,<dbl>,<int>
6,6.642287,2.812814,1
7,7.444542,3.162954,1


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,-2.637662,-25.480046,-2.518308,-4.468314,-4.298320,-18.642598,-3.038728,-6.595785,-2.170915,-2.041556,⋯,-10.699713,-6.262872,-19.297443,-3.938205,-10.230967,-3.733169,-5.536809,-9.015933,50.41,0
2,-2.609364,-20.936274,-2.493666,-4.148959,-3.461897,-12.900643,-2.626328,-4.074350,-2.068588,-1.980392,⋯,-5.956417,-6.238628,-14.452018,-3.888742,-6.810179,-3.621793,-5.273815,-8.923980,50.53,0
3,-2.580623,-15.016940,-2.468736,-3.996094,-3.301638,-12.015046,-2.480375,-4.067282,-1.942540,-1.940162,⋯,-5.775451,-6.220250,-11.690438,-3.674306,-3.961696,-3.582814,-4.850184,-8.758956,50.70,0
4,-2.551425,-14.821249,-2.443510,-3.920552,-3.254039,-9.726991,-2.452595,-4.049661,-1.935719,-1.929278,⋯,-5.696745,-6.192213,-9.277415,-3.651065,-3.875061,-3.448838,-4.677707,-7.623396,53.43,0
5,-2.543593,-14.605105,-2.417978,-3.868896,-3.206065,-7.780302,-2.424652,-4.018033,-1.928932,-1.920683,⋯,-5.670307,-6.173698,-8.428863,-3.619889,-3.745031,-3.430118,-4.641842,-7.262726,54.04,0
6,-2.521749,-11.294675,-2.393498,-3.823482,-3.157705,-6.907847,-2.396545,-4.007709,-1.927792,-1.918385,⋯,-5.649456,-6.151633,-6.799407,-3.516737,-3.523117,-3.410041,-4.618920,-6.837813,55.39,0
7,-2.513872,-11.123808,-2.361764,-3.809880,-3.108947,-5.805149,-2.368268,-3.984524,-1.922172,-1.913524,⋯,-5.577835,-4.363603,-6.620316,-3.470262,-3.386582,-3.398027,-4.461079,-6.830490,57.48,0
8,-2.490300,-9.442053,-2.343357,-3.796197,-3.080113,-4.902848,-2.351729,-3.959842,-1.915436,-1.907480,⋯,-5.572485,-3.484114,-5.895783,-3.372470,-3.364507,-3.361095,-4.399786,-6.826173,59.19,0
9,-2.466559,-8.544918,-2.316555,-3.662857,-2.968957,-4.816992,-2.319007,-3.943795,-1.911040,-1.896561,⋯,-5.568043,-3.367893,-5.775965,-3.335895,-3.297102,-3.302606,-4.308811,-6.330663,60.86,0


Ahora, podemos probar el modelo creado

In [30]:
creditcard_test <- data.frame(lapply(creditcard_test, sort))
split_creditcard2023_test_r1 <- creditcard_test[creditcard_test$V1 <
                                                  split_creditcard_2023$value, ]
split_creditcard2023_test_r2 <- creditcard_test[creditcard_test$V1 >=
                                                  split_creditcard_2023$value, ]
split_creditcard_test <- list(R1 = split_creditcard2023_test_r1,
                              R2 = split_creditcard2023_test_r2)
split_creditcard_test

accuracy_r1 <- sum(dim(split_creditcard2023_test_r1[split_creditcard2023_test_r1
                                                    $Class == 0, ])[1])
accuracy_r2 <- sum(dim(split_creditcard2023_test_r2[split_creditcard2023_test_r2
                                                    $Class == 1, ])[1])
accuracy <- (accuracy_r1 / dim(split_creditcard2023_test_r1)[1] +
               accuracy_r2 / dim(split_creditcard2023_test_r2)[1]) / 2
accuracy * 100

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,-2.735211,-11.006236,-2.600419,-4.140699,-3.450680,-6.402244,-2.681276,-3.397355,-2.740952,-2.515992,⋯,-3.938117,-3.047244,-11.273871,-3.671693,-4.530496,-3.807622,-4.809814,-14.861503,51.44,0
2,-2.707827,-9.750012,-2.578182,-4.088214,-3.422336,-5.615938,-2.660446,-2.309111,-2.598234,-2.334690,⋯,-3.042176,-2.902875,-10.270414,-3.567598,-4.304435,-3.420628,-3.458518,-8.902624,54.87,0
3,-2.686628,-9.279405,-2.549988,-4.002641,-3.316655,-4.839882,-2.636296,-1.984652,-2.547855,-2.157019,⋯,-1.823877,-2.753699,-8.710230,-3.381105,-3.282465,-3.414544,-3.276941,-6.525335,61.93,0
4,-2.184222,-8.544413,-1.954368,-3.869528,-2.943699,-4.285727,-2.572037,-1.901134,-2.536363,-2.103428,⋯,-1.782453,-2.527554,-6.957472,-3.177406,-2.747233,-3.399889,-3.127901,-6.378206,74.03,0
5,-2.086423,-8.231273,-1.943180,-3.850495,-2.560038,-3.869002,-2.562091,-1.851613,-2.424235,-2.089028,⋯,-1.766336,-2.438652,-6.081819,-3.175618,-2.563958,-3.330823,-3.113267,-5.867298,74.72,0
6,-2.034729,-8.180905,-1.863725,-3.772298,-2.496764,-3.305728,-2.249872,-1.591153,-2.380712,-2.048877,⋯,-1.742665,-2.203543,-5.425204,-3.168827,-2.547873,-3.324770,-3.080165,-5.299102,83.92,0
7,-1.930032,-7.887634,-1.814920,-3.618630,-2.481960,-3.229557,-2.116285,-1.513143,-2.372762,-2.034626,⋯,-1.730462,-2.043034,-5.104587,-3.140737,-2.542601,-3.227324,-2.917630,-5.141293,87.11,0
8,-1.852223,-7.494327,-1.748613,-3.569770,-2.421195,-2.797838,-2.060551,-1.512404,-2.322493,-1.982970,⋯,-1.681567,-1.933262,-4.969868,-2.761435,-2.537909,-3.144424,-2.897917,-4.610703,88.05,0
9,-1.840303,-6.909646,-1.714206,-3.504095,-2.325498,-2.728906,-1.985076,-1.467021,-2.317736,-1.972634,⋯,-1.641707,-1.924906,-4.061471,-2.748895,-2.502113,-3.087600,-2.881941,-4.606365,98.19,0


[1] 99.93964